In [ ]:
# 测试 IndexFlatL2
import faiss
import numpy as np

# Create index with dimension 5
index = faiss.IndexFlatL2(5)

# Create vectors as a numpy array (float32) with shape (3, 5)
vectors = np.array([
    [1, 0, 0, 0, 1],
    [1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1]
], dtype=np.float32)

# Add all vectors at once
index.add(vectors)

# Create query vector (must be 2D array)
query = np.array([[1, 1, 0, 0, 1],[1,1,1,1,1]], dtype=np.float32)

# 查询query中的临近向量
distances, labels = index.search(query, 1)

print("Distances:", distances)
print("Labels:", labels)

Distances: [[1.]
 [0.]]
Labels: [[0]
 [2]]


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="bge-large-zh-v1.5",
    cache_folder="./bge-large-zh-v1.5",
)
print("load embeddings successfully")

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

sample_embedding = embeddings.embed_query("hello world")
print(f'embeddings length is {len(sample_embedding)}')

# index是一个IndexFlatL2对象，用来存储向量索引
index = faiss.IndexFlatL2(len(sample_embedding))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

vector_store.save_local("faiss_index")

load embeddings successfully
embeddings length is 1024


In [ ]:
# 从本地加载faiss索引
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": {"$eq": "tweet"}},
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")


* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
